# Fiddling around with making Bungie API calls to get json data 

## [Error Codes](https://bungie-net.github.io/multi/schema_Exceptions-PlatformErrorCodes.html#schema_Exceptions-PlatformErrorCodes)
## [Manifest](https://bungie-net.github.io/multi/operation_get_Destiny2-GetDestinyManifest.html#operation_get_Destiny2-GetDestinyManifest)


In [209]:
import pandas as pd
import requests
import os

In [232]:
class BungieData(object):

    def __init__(self, api_key, session=None):
        '''
        session: an object from login.py defining a user session by 
        logging into Bungie.net (oAuth verified)
        '''
        self.session = session
        self.api_key = api_key
        
    def get_playerByTagName(self, gamertag):
        site_call = "https://bungie.net/Platform/Destiny2/SearchDestinyPlayer/2/" + gamertag
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_DestinyUserId(self, gamertag):
        info = self.get_playerByTagName(gamertag)
        return int(info[0]['membershipId'])

    def get_BungieUserId(self, membership_id):
        '''
        Use old Destiny endpoint for PSN user to get the BUNGIE membershipId for a user
        '''
        site_call = "https://bungie.net/Platform/User/GetMembershipsById/" + str(membership_id) + "/2/" 
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return int(request.json()['Response']['bungieNetUser']['membershipId'])
    
    def get_DestinyUserProfile(self, membership_id, components=[100]):
        '''
        Use new Destiny 2 endpoint for PSN player using the Destiny membershipId 
        '''
        components = "?components=" + ','.join([str(c) for c in components])
        site_call = "https://bungie.net/Platform/Destiny2/2/Profile/" + str(membership_id) + "/" + components
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_postGameStats(self, game_id):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(game_id)
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_Manifest(self):
        site_call = "https://bungie.net/Platform/Destiny2/Manifest/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_PlayerStats(self, membership_id):
        site_call = "https://bungie.net/Platform/Destiny2/2/Account/" + str(membership_id) + "/Stats/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']
    
    def get_StatDefinitions(self):
        site_call = "https://bungie.net/Platform/Destiny2/Stats/Definition/"
        request = requests.get(site_call, 
                                headers={"X-API-Key":self.api_key})
        return request.json()['Response']

In [233]:
bungie = BungieData(api_key=os.environ["BUNGIE_API_KEY"]) # Never put your keys in code... export 'em!

In [221]:
# Get my own Bungie MembershipID
Me = bungie.get_playerByTagName("BalancedSeeker6")
Me

[{u'displayName': u'BalancedSeeker6',
  u'iconPath': u'/img/theme/destiny/icons/icon_psn.png',
  u'membershipId': u'4611686018453385415',
  u'membershipType': 2}]

In [222]:
# Get Destiny MembershipID
my_destiny_id = bungie.get_DestinyUserId("BalancedSeeker6")
my_destiny_id

4611686018453385415

In [223]:
# Get my BUNGIE membershipID from using my DESTINY membershipID found in the above call
# This is the number found on the profile page of a user on Bungie.net
my_bungie_id = bungie.get_BungieUserId(4611686018453385415)
my_bungie_id

11927477

### To use GetProfile, you'll need to lookup the user's Destiny membershipID by searching by gamertag. Found that in the get_DestinyUserId call : 4611686018453385415

In [224]:
My_Profile = bungie.get_DestinyUserProfile(4611686018453385415, components=[100,200])
My_Profile 

{u'characters': {u'data': {u'2305843009260518141': {u'baseCharacterLevel': 20,
    u'characterId': u'2305843009260518141',
    u'classHash': 3655393761,
    u'classType': 0,
    u'dateLastPlayed': u'2017-09-30T07:06:14Z',
    u'emblemBackgroundPath': u'/common/destiny2_content/icons/c06afc4c698fbec7ee17e9d21496e192.jpg',
    u'emblemHash': 4077939641,
    u'emblemPath': u'/common/destiny2_content/icons/60530e4d21f5a32bebdca82cb178c4bf.jpg',
    u'genderHash': 2204441813,
    u'genderType': 1,
    u'levelProgression': {u'currentProgress': 146000,
     u'dailyLimit': 0,
     u'dailyProgress': 0,
     u'level': 20,
     u'levelCap': 20,
     u'nextLevelAt': 0,
     u'progressToNextLevel': 0,
     u'progressionHash': 1716568313,
     u'stepIndex': 20,
     u'weeklyLimit': 0,
     u'weeklyProgress': 0},
    u'light': 283,
    u'membershipId': u'4611686018453385415',
    u'membershipType': 2,
    u'minutesPlayedThisSession': u'116',
    u'minutesPlayedTotal': u'5120',
    u'percentToNextLeve

In [225]:
# Get a random single game's stats
game_stats = bungie.get_postGameStats(100)
game_stats

{u'activityDetails': {u'directorActivityHash': 1658347443,
  u'instanceId': u'100',
  u'isPrivate': False,
  u'mode': 2,
  u'modes': [7, 2],
  u'referenceId': 1658347443},
 u'entries': [{u'characterId': u'2305843009260533952',
   u'extended': {u'values': {u'precisionKills': {u'basic': {u'displayValue': u'55',
       u'value': 55.0}}},
    u'weapons': [{u'referenceId': 53159280,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'4',
         u'value': 4.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'75%',
         u'value': 0.75}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'displayValue': u'3',
         u'value': 3.0}}}},
     {u'referenceId': 4024037919,
      u'values': {u'uniqueWeaponKills': {u'basic': {u'displayValue': u'75',
         u'value': 75.0}},
       u'uniqueWeaponKillsPrecisionKills': {u'basic': {u'displayValue': u'65%',
         u'value': 0.6533333333333333}},
       u'uniqueWeaponPrecisionKills': {u'basic': {u'

In [226]:
manifest = bungie.get_Manifest()
manifest

{u'mobileAssetContentPath': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
 u'mobileClanBannerDatabasePath': u'/common/destiny2_content/clanbanner/clanbanner_sql_content_12a291b583d3f5ce5c30a648fa58b482.content',
 u'mobileGearAssetDataBases': [{u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_30228cabcdcf681095c36fab1da69e32.content',
   u'version': 0},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 1},
  {u'path': u'/common/destiny2_content/sqlite/asset/asset_sql_content_669cef97d6bb31f46d5703bcb9acb7bd.content',
   u'version': 2}],
 u'mobileGearCDN': {u'Gear': u'/common/destiny2_content/geometry/gear',
  u'Geometry': u'/common/destiny2_content/geometry/platform/mobile/geometry',
  u'PlateRegion': u'/common/destiny2_content/geometry/platform/mobile/plated_textures',
  u'Shader': u'/common/destiny2_content/geometry/platform/mobile/shad

In [227]:
My_Stats = bungie.get_PlayerStats(my_destiny_id)
My_Stats

{u'characters': [{u'characterId': u'2305843009289675361',
   u'deleted': False,
   u'merged': {u'allTime': {u'activitiesCleared': {u'basic': {u'displayValue': u'15',
       u'value': 15.0},
      u'statId': u'activitiesCleared'},
     u'activitiesEntered': {u'basic': {u'displayValue': u'40', u'value': 40.0},
      u'statId': u'activitiesEntered'},
     u'activitiesWon': {u'basic': {u'displayValue': u'1', u'value': 1.0},
      u'statId': u'activitiesWon'},
     u'adventuresCompleted': {u'basic': {u'displayValue': u'3', u'value': 3.0},
      u'pga': {u'displayValue': u'0.1', u'value': 0.075},
      u'statId': u'adventuresCompleted'},
     u'allParticipantsCount': {u'basic': {u'displayValue': u'186',
       u'value': 186.0},
      u'statId': u'allParticipantsCount'},
     u'allParticipantsScore': {u'basic': {u'displayValue': u'335',
       u'value': 335.0},
      u'statId': u'allParticipantsScore'},
     u'allParticipantsTimePlayed': {u'basic': {u'displayValue': u'2d 7h',
       u'value':

In [228]:
def char_stats_json2df(json):
    for char in My_Stats['characters']:
        if not char['deleted']:
            char_id = int(char['characterId'])

statsdf = pd.DataFrame(My_Stats)
statsdf

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [234]:
stat_definitions = bungie.get_StatDefinitions()
stat_definitions

{u'activitiesCleared': {u'category': 0,
  u'group': 1,
  u'modes': [7, 6, 4, 2, 3, 17, 16, 18],
  u'periodTypes': [2, 1],
  u'statDescription': u'Successful completions of this mission type.',
  u'statId': u'activitiesCleared',
  u'statName': u'Activities Cleared',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activitiesEntered': {u'category': 8,
  u'group': 1,
  u'modes': [5, 10, 12, 7, 6, 4, 2, 3, 17, 16, 18, 19, 37, 38, 31, 39],
  u'periodTypes': [2, 1],
  u'statDescription': u'Battles fought - some lost, some won.',
  u'statId': u'activitiesEntered',
  u'statName': u'Activities Entered',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activitiesWon': {u'category': 0,
  u'group': 1,
  u'modes': [5, 10, 12, 31, 19, 38, 37, 39],
  u'periodTypes': [2, 1],
  u'statId': u'activitiesWon',
  u'statName': u'Wins',
  u'unitLabel': u'',
  u'unitType': 1,
  u'weight': 1},
 u'activityAssists': {u'category': 2,
  u'group': 102,
  u'modes': [5, 10, 12, 7, 6, 4, 2, 3, 17, 1